In [1]:
1+1

2

In [3]:
import os
from os.path import join as pjoin

print("Starting to read from folder prescription/")
data_dir = "prescription/"
dirs = [pjoin(data_dir,directory) for directory in os.listdir(data_dir) if os.path.isdir(pjoin(data_dir,directory))]
xmls = [[pjoin(dir_name,fn) for fn in os.listdir(dir_name) if fn.endswith(".xml")][0] for dir_name in dirs]

Starting to read from folder prescription/


- [ ] Recuperer l'active moiety dans la balise manufacturedMedicine
- [ ] Recuperer le strength de la substance sous la balise quantity -> numerator et denominator
- [ ] Recuperer l'active ingredient pour le format 2 (manufacturedProduct/manufacturedProduct) par la balise ingredientSubstance/name et l'active moiety par activeMoiety (seulement si la balise contient activeMoiety)

In [4]:
from lxml import etree as et
import re
import lxml

from lxml.etree import XMLParser, parse

def clean_string(string):
#     return re.sub('[\t\n]','',string.lower().strip())
    return string.upper().strip()
def t(tagname):
    ns = '{urn:hl7-org:v3}'
    return ns + tagname

def strip_namespace(root):
    ns = '{urn:hl7-org:v3}'
    root.tag.replace(ns,'')
    for elem in root.iter():  
        elem.tag = elem.tag.replace(ns, '')
        
def get_elements(xml_file):
    try:
        _, author_element = next(et.iterparse(xml_file, tag=t("author")))
        strip_namespace(author_element)
    except StopIteration:
        author_element = None
    _, subject_element = next(et.iterparse(xml_file,tag=t("subject")))
    _, subject_element = next(et.iterparse(xml_file,tag=t("subject")))
    strip_namespace(subject_element)
    return author_element, subject_element


def safetify(fn,ret_val=None):
    def wrapper(el):
        try:
            return fn(el)
        except Exception as e:
            print(e)
            return ret_val
    return wrapper
def return_merged(elems,**kwargs):
    return [{name: safetify(fn)(elem) for name,fn in kwargs.items()} for elem in elems]

def get_info(xml_file,author_element,subject_element,pick_ings=True):
    author_name = "NA" if author_element is None else author_element.xpath(("assignedEntity//name"))[0].text
    drug_names = subject_element.xpath(".//manufacturedMedicine/name")
   
    if len(drug_names) < 1:
        return None
    code = subject_element.xpath(".//manufacturedMedicine/code")[0].attrib['code']
    drug_name = drug_names[0].text
    active_ingredients_elem = subject_element.xpath(".//activeMoiety/activeMoiety")
#     import ipdb; ipdb.set_trace()
    try:
        def get_text(elem):
            return clean_string(elem.xpath("name")[0].text)
        def get_code(elem):
            return elem.xpath("code")[0].attrib['code']
        codes_and_texts = return_merged(active_ingredients_elem,name=get_text,code=get_code)
        return dict(author=author_name,drug=drug_name,active_ingredients=codes_and_texts,code=code,filename=xml_file)
    except ValueError as e:
        import ipdb; ipdb.set_trace()
        print('exception here: '+ str(e))
        return dict()
#     codes = [elem.xpath("code")[0].attrib['code'] for elem in active_ingredients_elem]
#     texts = [elem.xpath("name")[0].text for elem in active_ingredients_elem]
#     active_ingredient = ' ||| '.join(texts)
#     active_ingredient_code = ' ||| '.join(codes)
   

    
def get_info_alt(xml_file,author_element,subject_element,pick_ings=True):
    author_name = "NA" if author_element is None else author_element.xpath("assignedEntity//name")[0].text
    drug_name = subject_element.xpath(".//manufacturedProduct/manufacturedProduct/name")[0].text
    active_ingredients_elem = subject_element.xpath(".//activeMoiety/activeMoiety")
#     active_ingredients_code = subject_element.xpath("//activeMoiety/code")
    if len(active_ingredients_elem) < 1:
        return None
    
    
    get_code = lambda elem: elem.xpath("code")[0].attrib['code']
    get_text = lambda elem: elem.xpath("name")[0].text
    codes_and_texts = return_merged(active_ingredients_elem,name=get_text,code=get_code)
    
#     active_ingredient = ' ||| '.join(texts)
    active_ingredient = codes_and_texts
#     active_ingredient_code = ' ||| '.join(codes)
    code = subject_element.xpath(".//manufacturedProduct/manufacturedProduct/code")[0].attrib['code']
    return dict(author=author_name,drug=drug_name,active_ingredient=active_ingredient,code=code,filename=xml_file)

def get_form(subject_element):
    form_code_elem = subject_element.xpath(".//manufacturedProduct//formCode")[0]
    try:
        code = form_code_elem.attrib['code']
        form = clean_string(form_code_elem.attrib['displayName'])
        form_list = [{'name' : form, 'code' : code}]
        return dict(form=form_list)
    except Exception as e:
        print("Error occured ",e)
        return dict(form=[])
patt = re.compile('[0-9]+\.[0-9]+')
regex = re.compile(r'<[^>]+>')
def xml_to_string(el):
    return '\n'.join([el for el in regex.sub('',et.tostring(el).decode('utf-8')).split() if len(el) > 1])
def le(alist,length=1):
    return len(alist)==length

def read_elem_for_strength(strength_elem):
    vals = ['displayName','value']
    return [strength_elem.attrib[val] for val in vals]

def get_strength(subject_element):
    try:

        strength_elem_numerator = subject_element.xpath(".//quantity/numerator/translation")[0]
        strength_elem_denominator = subject_element.xpath(".//quantity/denominator/translation")[0]
        numerator_name, numerator_val = read_elem_for_strength(strength_elem_numerator)
        denominator_name, denominator_val = read_elem_for_strength(strength_elem_denominator)
        return {"numerator_name" : numerator_name, "numerator_val" : numerator_val, "denominator_name": denominator_name, "denominator_val" : denominator_val}
    except Exception as e:
#         print(f"An error occured: {e}")
        return {}
def get_adverse_events(xml_path):
    adverse_els = []
    for _, element in et.iterparse(xml_path,tag=t("component")):
        strip_namespace(element)
        if len(element.xpath("section/text/content")) > 0:
            text = element.xpath("section/text/content")[0].text
            if "ADVERSE" in text and "REACTION" in text:
                return {"adverse_events_list" : [], "adverse_events" : tostring(element)}
        if len(element.xpath("section/title")) == 0:
            continue
        if element.xpath("section/title")[0].text is None:
            continue
        if len(element.xpath("section/code")) > 0 and 'ADVERSE REACTIONS' in element.xpath("section/title")[0].text:
            adverse_reaction_els = element.xpath("section/text/table/tbody/tr")
#             import ipdb; ipdb.set_trace()
            if len(adverse_reaction_els) > 1:      
                for adverse_reaction_el in adverse_reaction_els:
                    if len(adverse_reaction_el.xpath("td")) == 2 and le(adverse_reaction_el.xpath("td/content")):
                        t1 = adverse_reaction_el.xpath("td/content")[0].text
                        t2 = adverse_reaction_el.xpath("td")[1].text
                        if patt.match(t2):
                            adverse_els.append([t1,t2])
            elif len(element.xpath("section/component/section/text/list/item")) > 1:
                adverse_els = [[tostring(el),""] for el in element.xpath("section/component/section/text/list/item")]
        
            
            return {"adverse_events_list" : [el[0] for el in adverse_els if valid_line(el[0])], "adverse_events" : tostring(element)}
    return {"adverse_events_list" : [], "adverse_events" : ""}
space_patt = re.compile("\W")
def valid_line(line):
#     print(f"Line is: {line}.")
    return len(space_patt.sub('',line)) > 2

def tostring(element):
    return '\n'.join([line for line in element.itertext() if valid_line(line)])

def get_indications(xml_path):
#     indication_text = []
    for _, element in et.iterparse(xml_path,tag=t("component")):
        strip_namespace(element)
        if len(element.xpath("section/text/content")) > 0:
            title = element.xpath("section/text/content")[0].text
            if "INDICATIONS" in title and "USAGE" in title:
                return tostring(element)
        if len(element.xpath("section/title")) == 0:
            continue
        if element.xpath("section/title")[0].text is None:
            continue
        if len(element.xpath("section/code")) > 0 and 'INDICATIONS AND USAGE' in element.xpath("section/title")[0].text:
            return tostring(element)
    return ""
    
'INDICATIONS AND USAGE'
'ADVERSE REACTIONS'
def get_roa(subject_element):
    try:
        roa_elems = subject_element.xpath(".//consumedIn//routeCode")
        get_name = lambda elem: clean_string(elem.attrib['displayName'])
        get_code = lambda elem: elem.attrib['code']
        roas = return_merged(roa_elems,name=get_name,code=get_code)
        return dict(roa=roas)
    except Exception as e:
        print("Error occured ",e)
        return dict(roa="NA",roa_code="NA")

def has_no_active_ingredients(info,key='active_ingredient'):
    if key in info:
        return any(map(lambda myd: any(filter(lambda el: el is not None,myd.values())), test))
    return True


def get_info_from_xml(xml_file):
    author_elem, subject_elem = get_elements(xml_file)
    info = get_info(xml_file,author_elem,subject_elem)
#     import ipdb; ipdb.set_trace()
    
    if info is None:
        info = get_info_alt(xml_file,author_elem,subject_elem)
    form_dict = get_form(subject_elem)
    roa_dict = get_roa(subject_elem)
    strength_dict = get_strength(subject_elem)
#     if len(strength_dict) == 0:
#         print(f"Cannot read the strength from this file: {xml_file}")
    adverse_elems = {"adverse_events" : "","adverse_events_list" : []}
#     strength_tags = get_strength_tags(subject_elem)
    indications_elems = ""
    try:
        adverse_elems = get_adverse_events(xml_file)
        indications_elems = get_indications(xml_file)
    except Exception as e:
        pass
    complete_dict = {**info,**form_dict,**roa_dict, **strength_dict, **adverse_elems, "indications" : indications_elems}
    return complete_dict

                                        
# /home/james/Datasets/dailymed/prescription/20060131_dffb4544-0e47-40cd-9baa-d622075838cc/dffb4544-0e47-40cd-9baa-d622075838cc.xml
                                          

test_fn = ('prescription/20060131_dffb4544-0e47-40cd-9baa-d622075838cc/dffb4544-0e47-40cd-9baa-d622075838cc.xml')
get_info_from_xml(test_fn)

{'author': 'Celltech Pharmaceuticals, Inc.',
 'drug': 'Mykrox Tablets',
 'active_ingredients': [{'name': 'METOLAZONE', 'code': 'TZ7V40X7VX'}],
 'code': '53014-847',
 'filename': 'prescription/20060131_dffb4544-0e47-40cd-9baa-d622075838cc/dffb4544-0e47-40cd-9baa-d622075838cc.xml',
 'form': [{'name': 'TABLET', 'code': 'C42998'}],
 'roa': [{'name': 'ORAL', 'code': 'C38288'}],
 'numerator_name': 'MICROGRAM',
 'numerator_val': '500',
 'denominator_name': 'TABLET',
 'denominator_val': '1',
 'adverse_events_list': ['Dizziness (lightheadedness)',
  'Headaches ',
  'Muscle Cramps',
  'Fatigue (malaise, lethargy, lassitude)',
  'Joint Pain, swelling',
  'Chest Pain (precordial discomfort)'],
 'adverse_events': 'ADVERSE REACTIONS\nAdverse experience information is available from more than 14 years of accumulated marketing experience with other formulations of metolazone for which reliable quantitative information is lacking and from controlled clinical trials with MYKROX from which incidences can

In [14]:
%pdb on
# from tqdm.notebook import tqdm
def extract_strength_from_active_ingredient_tag(tag):
#     import ipdb; ipdb.set_trace()
    num_tag = tag.xpath('quantity/numerator')[0].attrib
    denom_tag = tag.xpath('quantity/denominator')[0].attrib
#     num_unit_val = None
    if 'unit' not in num_tag:
        translation_tags = tag.xpath('quantity/numerator/translation')
        if len(translation_tags) == 0:
            import ipdb; ipdb.set_trace()
        else:
            num_tag['unit'] = translation_tags[0].attrib['displayName']
    
    nums = ' '.join([num_tag['value'],num_tag['unit']])
    denoms = ''
    if 'unit' in denom_tag:
        denoms = ' '.join([denom_tag['value'],denom_tag['unit']])
    name = tag.xpath(".//name")[0].text
    code = tag.xpath('.//code')[0].attrib['code']
    codesystem = tag.xpath('.//code')[0].attrib['codeSystem']
    full_code = f"{code} ||| {codesystem}"
    label = nums if 'unit' not in denom_tag or denom_tag['unit'] == '1' else nums + ' / ' + denoms
    return name, full_code, label
def parse_active_ingredient_tag(tag):
    ing_tags = get_strength_tags(tag)
    compos = [extract_strength_from_active_ingredient_tag(tag) for tag in ing_tags]
    return compos
def get_strength_tags(subject_element):
    quantity_tags = subject_element.xpath(".//activeIngredient")
    if len(quantity_tags) == 0:
#         import ipdb; ipdb.set_trace()
        quantity_tags = [el for el in subject_element.xpath('.//ingredient') if el.attrib['classCode'].startswith('ACTI')]
#         import ipdb; ipdb.set_trace()
#     import ipdb; ipdb.set_trace()
    quants = [(quantity_tag) for quantity_tag in quantity_tags]
    return quants
 

def parse_xml_with_strength(xml_file):
    author_elem, subject_elem = get_elements(xml_file)
#     info = get_info(xml_file,author_elem,subject_elem,False)
# #     import ipdb; ipdb.set_trace()
    
#     if info is None:
#         info = get_info_alt(xml_file,author_elem,subject_elem,False)
#     form_dict = get_form(subject_elem)
#     roa_dict = get_roa(subject_elem)
    compos = parse_active_ingredient_tag(subject_elem)
    return compos


all_ings = [(xml_file, (parse_xml_with_strength)(xml_file)) for xml_file in (xmls)]

Automatic pdb calling has been turned ON


In [16]:
all_ings

[('prescription/20180315_c89388f0-ffe4-4058-a644-3f43fb4d021b/dd233955-5be5-4025-a841-66378e261dd2.xml',
  [('ETHINYL ESTRADIOL', '423D2T571U ||| 2.16.840.1.113883.4.9', '0.025 mg'),
   ('NORETHINDRONE', 'T18F433X4S ||| 2.16.840.1.113883.4.9', '0.8 mg')]),
 ('prescription/20201229_007a6ad8-4d23-4543-ae48-c651cac7dd7b/30462459-24c1-407e-a045-556e1938242a.xml',
  [('ENTECAVIR', '5968Y6H45M ||| 2.16.840.1.113883.4.9', '0.5 mg')]),
 ('prescription/20120920_5dcd492f-cef1-41ba-b5d2-c79c31601f63/ecdd65cd-25d6-401a-b925-fb1652999167.xml',
  [('POTASSIUM BICARBONATE',
    'HM5Z15LEBN ||| 2.16.840.1.113883.4.9',
    '25 meq')]),
 ('prescription/20210819_a9c86f3a-62f7-418c-e053-2a95a90a429f/a9c88a41-3ff6-78f7-e053-2995a90adb8f.xml',
  [('LABETALOL HYDROCHLORIDE',
    '1GEV3BAW9J ||| 2.16.840.1.113883.4.9',
    '100 mg')]),
 ('prescription/20220412_47423aa5-974e-4c52-8d51-a3a3a71e3260/dc64ef66-bb98-973a-e053-2995a90ad8bc.xml',
  [('IRBESARTAN', 'J0E2756Z7N ||| 2.16.840.1.113883.4.9', '300 mg'),
  

In [17]:
get_strength_tags_from_xml(xmls[0])

[('ETHINYL ESTRADIOL', '423D2T571U ||| 2.16.840.1.113883.4.9', '0.025 mg'),
 ('NORETHINDRONE', 'T18F433X4S ||| 2.16.840.1.113883.4.9', '0.8 mg')]

In [18]:
xmls[0]

'prescription/20180315_c89388f0-ffe4-4058-a644-3f43fb4d021b/dd233955-5be5-4025-a841-66378e261dd2.xml'

In [19]:
import pickle
with open("dailymed_all_drugs.pickle","wb") as f:
    pickle.dump(all_ings,f)

In [12]:
1+1

2

In [29]:
print(et.tostring(all_ings[0][0]).decode('utf-8'))

<ingredient xmlns="urn:hl7-org:v3" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" classCode="ACTIB">
                                 <quantity>
                                    <numerator unit="mg" value="0.025"/>
                                    <denominator unit="1" value="1"/>
                                 </quantity>
                                 <ingredientSubstance>
                                    <code code="423D2T571U" codeSystem="2.16.840.1.113883.4.9"/>
                                    <name>ETHINYL ESTRADIOL</name>
                                    <activeMoiety>
                                       <activeMoiety>
                                          <code code="423D2T571U" codeSystem="2.16.840.1.113883.4.9"/>
                                          <name>ETHINYL ESTRADIOL</name>
                                       </activeMoiety>
                                    </activeMoiety>
                                 </ingredientSubstanc

In [23]:
xmls[39954]

'prescription/20180731_741d022f-954f-4686-b60e-08337235e9a7/59b669c4-f24a-4d5b-8026-31638935f599.xml'

In [21]:
from collections import Counter
[i for i, el in enumerate(all_ings) if len(el) == 0]

[39954]

In [24]:
from collections import Counter
Counter([len(el) for el in all_ings])

Counter({2: 4121,
         1: 41099,
         17: 26,
         3: 629,
         4: 303,
         7: 19,
         6: 105,
         16: 20,
         13: 22,
         8: 44,
         5: 113,
         9: 14,
         11: 12,
         21: 4,
         10: 15,
         12: 14,
         15: 26,
         20: 6,
         19: 4,
         14: 13,
         18: 17,
         22: 1,
         23: 1,
         30: 2,
         24: 6,
         28: 1,
         25: 1,
         0: 1,
         31: 1})

In [4]:
test = [{'name': 'METOLAZONE', 'code': 'TZ7V40X7VX'}]
any(map(lambda myd: any(filter(lambda el: el is not None,myd.values())), test))

True

In [5]:
get_adverse_events("prescription/20180315_c89388f0-ffe4-4058-a644-3f43fb4d021b/dd233955-5be5-4025-a841-66378e261dd2.xml")

{'adverse_events_list': ['Serious cardiovascular events and smoking \n[see \nBoxed Warning\n, and \nWarnings and Precautions (5.1)',
  'Vascular events \n[see \nWarnings and Precautions (5.1)',
  'Liver disease \n[see \nWarnings and Precautions (5.3)',
  'Irregular uterine bleeding',
  'Nausea ',
  'Breast tenderness',
  'Headache '],
 'adverse_events': '6 ADVERSE REACTIONS\nThe most common adverse reactions (≥ 2%) are nausea/vomiting (8.8%), headaches/migraine (7.5%), depression/mood complaints (4.1%), dysmenorrhea (3.9%), acne (3.2%), anxiety symptoms (2.4%), breast pain/tenderness (2.4%), and increased weight (2.3%). (\nTo report SUSPECTED ADVERSE REACTIONS, contact Mylan Pharmaceutical Inc. at 1-877-446-3679 (1-877-4-INFO-RX) or FDA at 1-800-FDA-1088 or www.fda.gov/medwatch. \nThe following serious adverse reactions with the use of COCs are discussed elsewhere in the labeling: \nSerious cardiovascular events and smoking \n[see \nBoxed Warning\n, and \nWarnings and Precautions (5.1)

In [6]:
get_info_from_xml("prescription/20180315_c89388f0-ffe4-4058-a644-3f43fb4d021b/dd233955-5be5-4025-a841-66378e261dd2.xml")

{'author': 'Mylan Pharmaceuticals Inc.',
 'drug': 'NORETHINDRONE AND ETHINYL ESTRADIOL AND FERROUS FUMARATE',
 'active_ingredient': [{'name': 'ETHINYL ESTRADIOL', 'code': '423D2T571U'},
  {'name': 'NORETHINDRONE', 'code': 'T18F433X4S'}],
 'code': '0378-7308',
 'filename': 'prescription/20180315_c89388f0-ffe4-4058-a644-3f43fb4d021b/dd233955-5be5-4025-a841-66378e261dd2.xml',
 'form': [{'name': 'KIT', 'code': 'C47916'}],
 'roa': [{'name': 'ORAL', 'code': 'C38288'},
  {'name': 'ORAL', 'code': 'C38288'}],
 'adverse_events_list': ['Serious cardiovascular events and smoking \n[see \nBoxed Warning\n, and \nWarnings and Precautions (5.1)',
  'Vascular events \n[see \nWarnings and Precautions (5.1)',
  'Liver disease \n[see \nWarnings and Precautions (5.3)',
  'Irregular uterine bleeding',
  'Nausea ',
  'Breast tenderness',
  'Headache '],
 'adverse_events': '6 ADVERSE REACTIONS\nThe most common adverse reactions (≥ 2%) are nausea/vomiting (8.8%), headaches/migraine (7.5%), depression/mood comp

In [7]:
get_info_from_xml("prescription/20120920_5dcd492f-cef1-41ba-b5d2-c79c31601f63/ecdd65cd-25d6-401a-b925-fb1652999167.xml")

{'author': 'Qualitest Pharmaceuticals',
 'drug': 'K-EFFERVESCENT ',
 'active_ingredient': [{'name': 'POTASSIUM CATION', 'code': '295O53K152'}],
 'code': '0603-4170',
 'filename': 'prescription/20120920_5dcd492f-cef1-41ba-b5d2-c79c31601f63/ecdd65cd-25d6-401a-b925-fb1652999167.xml',
 'form': [{'name': 'TABLET, EFFERVESCENT', 'code': 'C42910'}],
 'roa': [{'name': 'ORAL', 'code': 'C38288'}],
 'adverse_events_list': [],
 'adverse_events': 'ADVERSE REACTIONS:\n\xa0 Abdominal discomfort, diarrhea, vomiting, and nausea are the most common adverse reactions tooral potassium supplements.\xa0 These adverse reactions occur most frequently when the preparation is not taken with food, is not properly diluted, or not completely dissolved.\nHyperkalemia occurs infrequently in patients with normal renal function who receive oral potassium supplements.\xa0 The following are symptoms of hyperkalemia: mental confusion, unexplained anxiety, cardiac arrhythmias, numbness or tingling in lips, feet, or hands,

In [8]:
get_indications(test_fn)

'INDICATIONS AND USAGE\nMYKROX Tablets are indicated for the treatment of hypertension, alone or in combination with other antihypertensive drugs of a different class. \nMYKROX TABLETS HAVE \nNOT\n BEEN EVALUATED FOR THE TREATMENT OF CONGESTIVE HEART FAILURE OR FLUID RETENTION DUE TO RENAL OR HEPATIC DISEASE AND THE CORRECT DOSAGE FOR THESE CONDITIONS AND OTHER EDEMA STATES HAS NOT BEEN ESTABLISHED. \nSINCE A SAFE AND EFFECTIVE \nDIURETIC\n DOSE HAS NOT BEEN ESTABLISHED, MYKROX TABLETS SHOULD \nNOT\n BE USED WHEN DIURESIS IS DESIRED. \nUsage in Pregnancy\nThe routine use of diuretics in an otherwise healthy woman is inappropriate and exposes mother and fetus to unnecessary hazard. Diuretics do not prevent development of toxemia of pregnancy, and there is no evidence that they are useful in the treatment of developed toxemia (see \nPRECAUTIONS\nEdema during pregnancy may arise from pathologic causes or from the physiologic and mechanical consequences of pregnancy. MYKROX is not indicate

In [9]:
get_adverse_events(test_fn)

{'adverse_events_list': ['Dizziness (lightheadedness)',
  'Headaches ',
  'Muscle Cramps',
  'Fatigue (malaise, lethargy, lassitude)',
  'Joint Pain, swelling',
  'Chest Pain (precordial discomfort)'],
 'adverse_events': 'ADVERSE REACTIONS\nAdverse experience information is available from more than 14 years of accumulated marketing experience with other formulations of metolazone for which reliable quantitative information is lacking and from controlled clinical trials with MYKROX from which incidences can be calculated. \nIn controlled clinical trials with MYKROX, adverse experiences resulted in discontinuation of therapy in 6.7-6.8% of patients given ½ to 1 mg of MYKROX.\nAdverse experiences occurring in controlled clinical trials with MYKROX with an incidence of >2%, whether or not considered drug-related, are summarized in the following table.\nIncidence of Adverse Experiences Volunteered or Elicited (by Patient in Percent)*\n* Percent of patients reporting an adverse experience on

# XML cases
## Case 1: tag "manufacturedMedicine" and "activeIngredientSubstance"
The first XML uses the tag manufacturedMedicine as well as activeIngredientSubstance to list the drug name and its active ingredients. There are only around 80 XML files that follow this specification.

Example: `prescription/20080319_F4FA5E60-5F70-46D7-8E29-DFCF8B783F2C/F4FA5E60-5F70-46D7-8E29-DFCF8B783F2C.xml`
## Case 2: tag "manufacturedProduct" with "activeMoiety"
The second case of XML has the tag manufacturedProduct and a tag partProduct that contains a list of activeMoiety tags, which have the active ingredients. There are around 15000 XML files that follow this specification.

Example: `prescription/20201229_007a6ad8-4d23-4543-ae48-c651cac7dd7b/30462459-24c1-407e-a045-556e1938242a.xml`
## Case 3: 
They may not contain tags about the author. In this case, we just extract the rest of the information (i.e. drug name and active ingredients following either case 1 or case 2 format)

Example: `prescription/20071212_EFCDB321-832B-4554-8049-DBBB33B48334/EFCDB321-832B-4554-8049-DBBB33B48334.xml`

In [10]:
get_info_from_xml("prescription/20200929_d07d9ae1-65a9-43f6-af84-896c0e7f7d69/b061d3e7-51c6-048c-e053-2995a90af53f.xml")

{'author': 'REMEDYREPACK INC.',
 'drug': 'Prenatal Vitamins Plus Low Iron',
 'active_ingredient': [{'name': 'VITAMIN A', 'code': '81G40H8B0T'},
  {'name': 'BETA CAROTENE', 'code': '01YAE03M7J'},
  {'name': 'ASCORBIC ACID', 'code': 'PQ6CK8PD0R'},
  {'name': 'CHOLECALCIFEROL', 'code': '1C6V77QF41'},
  {'name': '.ALPHA.-TOCOPHEROL, DL-', 'code': '7QWA1RIO01'},
  {'name': 'THIAMINE ION', 'code': '4ABT0J945J'},
  {'name': 'RIBOFLAVIN', 'code': 'TLM2976OFR'},
  {'name': 'NIACIN', 'code': '2679MF687A'},
  {'name': 'PYRIDOXINE', 'code': 'KV2JZ1BI6Z'},
  {'name': 'FOLIC ACID', 'code': '935E97BOY8'},
  {'name': 'CYANOCOBALAMIN', 'code': 'P6YC3EG204'},
  {'name': 'CALCIUM CATION', 'code': '2M83C4R6ZB'},
  {'name': 'IRON', 'code': 'E1UOL152H7'},
  {'name': 'ZINC', 'code': 'J41CSQ7QDS'},
  {'name': 'COPPER', 'code': '789U1901C5'}],
 'code': '70518-2746',
 'filename': 'prescription/20200929_d07d9ae1-65a9-43f6-af84-896c0e7f7d69/b061d3e7-51c6-048c-e053-2995a90af53f.xml',
 'form': [{'name': 'TABLET', '

In [10]:
# !code prescription/20201229_007a6ad8-4d23-4543-ae48-c651cac7dd7b/30462459-24c1-407e-a045-556e1938242a.xml

# !code prescription/20080319_F4FA5E60-5F70-46D7-8E29-DFCF8B783F2C/F4FA5E60-5F70-46D7-8E29-DFCF8B783F2C.xml

# !code prescription/20071212_EFCDB321-832B-4554-8049-DBBB33B48334/EFCDB321-832B-4554-8049-DBBB33B48334.xml

In [11]:
get_info_from_xml('prescription/20220709_8a5edcf9-828c-4f97-b671-268ab13a8ecd/1e2afa2f-2bc7-481c-a1f6-97632642199f.xml')

{'author': 'Indivior Inc.',
 'drug': 'Suboxone',
 'active_ingredient': [{'name': 'buprenorphine', 'code': '40D3SCR4GZ'},
  {'name': 'naloxone', 'code': '36B82AMQ7N'}],
 'code': '12496-1202',
 'filename': 'prescription/20220709_8a5edcf9-828c-4f97-b671-268ab13a8ecd/1e2afa2f-2bc7-481c-a1f6-97632642199f.xml',
 'form': [{'name': 'FILM, SOLUBLE', 'code': 'C42984'}],
 'roa': [{'name': 'SUBLINGUAL', 'code': 'C38300'},
  {'name': 'BUCCAL', 'code': 'C38193'}],
 'adverse_events_list': [],
 'adverse_events': '6 ADVERSE REACTIONS\n\nThe following serious adverse reactions are described elsewhere in the labeling:\nAddiction, Abuse, and Misuse [\nsee Warnings and Precautions (\nRespiratory and CNS Depression \n[see Warnings and Precautions (\nNeonatal Opioid Withdrawal Syndrome \n[see Warnings and Precautions (\nAdrenal Insufficiency \n[see Warnings and Precautions (\nOpioid Withdrawal \n[see Warnings and Precautions (\n5.10\nHepatitis, Hepatic Events \n[see Warnings and Precautions (\nHypersensitivi

In [12]:
from multiprocessing import cpu_count
from multiprocessing.pool import ThreadPool
from joblib import Parallel, delayed
def multithread_with_progress(fn,args_list,pool=ThreadPool(15)):
    total_len = len(args_list)
    def wrapper_fn(tup):
        index, *args = tup
        pct = (index / total_len) * 100
        if index % 1000 == 0:
            print(f"[{index}/{total_len}] ({pct:.2f}%)")
        return fn(*args)
    results = Parallel(n_jobs=-1)(delayed(wrapper_fn)(val) for val in enumerate(args_list))
    return results

In [13]:
from tqdm import tqdm
# results = multithread_with_progress(safetify(get_info_from_xml),xmls)
results = [safetify(get_info_from_xml)(xml_f) for xml_f in (xmls)]

In [14]:
[xml for result, xml in zip(results,xmls) if result is None]

['prescription/20180731_741d022f-954f-4686-b60e-08337235e9a7/59b669c4-f24a-4d5b-8026-31638935f599.xml']

In [15]:
clean_results = [result for result in results if result is not None]

In [16]:
len(clean_results)

46639

In [17]:
import pandas as pd
df = pd.DataFrame(clean_results)

In [18]:
df.to_csv("all_drugs_ade_indications_updated.csv")

In [19]:
len([el for el in df['adverse_events_list'].values if len(el) != 0])

3185

In [20]:
df['filename'].values[0]

'prescription/20180315_c89388f0-ffe4-4058-a644-3f43fb4d021b/dd233955-5be5-4025-a841-66378e261dd2.xml'

In [21]:
import pandas as pd
df = pd.read_csv("all_drugs_ade_indications_updated.csv")

In [22]:
df

,Unnamed: 0,author,drug,active_ingredient,code,filename,form,roa,adverse_events_list,adverse_events,indications,active_ingredients,numerator_name,numerator_val,denominator_name,denominator_val
0,0,Mylan Pharmaceuticals Inc.,NORETHINDRONE AND ETHINYL ESTRADIOL AND FERROU...,"[{'name': 'ETHINYL ESTRADIOL', 'code': '423D2T...",0378-7308,prescription/20180315_c89388f0-ffe4-4058-a644-...,"[{'name': 'KIT', 'code': 'C47916'}]","[{'name': 'ORAL', 'code': 'C38288'}, {'name': ...",['Serious cardiovascular events and smoking \n...,6 ADVERSE REACTIONS\nThe most common adverse r...,1 INDICATIONS AND USAGE\nNorethindrone and eth...,NaN,NaN,NaN,NaN,NaN
1,1,Amneal Pharmaceuticals LLC,Entecavir,"[{'name': 'ENTECAVIR ANHYDROUS', 'code': 'NNU2...",65162-446,prescription/20201229_007a6ad8-4d23-4543-ae48-...,"[{'name': 'TABLET, FILM COATED', 'code': 'C429...","[{'name': 'ORAL', 'code': 'C38288'}]",[],6 ADVERSE REACTIONS\nThe following adverse rea...,1 INDICATIONS AND USAGE\nEntecavir tablets are...,NaN,NaN,NaN,NaN,NaN
2,2,Qualitest Pharmaceuticals,K-EFFERVESCENT,"[{'name': 'POTASSIUM CATION', 'code': '295O53K...",0603-4170,prescription/20120920_5dcd492f-cef1-41ba-b5d2-...,"[{'name': 'TABLET, EFFERVESCENT', 'code': 'C42...","[{'name': 'ORAL', 'code': 'C38288'}]",[],"ADVERSE REACTIONS:\n Abdominal discomfort, di...",INDICATIONS AN USAGE:\n1. For therapeutic use ...,NaN,NaN,NaN,NaN,NaN
3,3,"Denton Pharma, Inc. dba Northwind Pharmaceuticals",Labetalol Hydrochloride,"[{'name': 'LABETALOL', 'code': 'R5H8897N95'}]",70934-751,prescription/20210819_a9c86f3a-62f7-418c-e053-...,"[{'name': 'TABLET, FILM COATED', 'code': 'C429...","[{'name': 'ORAL', 'code': 'C38288'}]",[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,REMEDYREPACK INC.,Irbesartan and Hydrochlorothiazide,"[{'name': 'IRBESARTAN', 'code': 'J0E2756Z7N'},...",70518-2619,prescription/20220412_47423aa5-974e-4c52-8d51-...,"[{'name': 'TABLET, FILM COATED', 'code': 'C429...","[{'name': 'ORAL', 'code': 'C38288'}]",[],6 ADVERSE REACTIONS\nMost common adverse event...,1 INDICATIONS AND USAGE\nIrbesartan and hydroc...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46634,46634,"EMD Serono, Inc.",Cetrotide,"[{'name': 'CETRORELIX', 'code': 'OON1HFZ4BA'}]",44087-1225,prescription/20220205_aca7768e-28a7-4027-b1d8-...,"[{'name': 'KIT', 'code': 'C47916'}]","[{'name': 'SUBCUTANEOUS', 'code': 'C38299'}, {...",[],ADVERSE REACTIONS\nThe safety of Cetrotide\n (...,INDICATIONS AND USAGE\nCetrotide\n (cetrorelix...,NaN,NaN,NaN,NaN,NaN
46635,46635,"Aphena Pharma Solutions - Tennessee, LLC",Mirtazapine,"[{'name': 'MIRTAZAPINE', 'code': 'A051Q2099Q'}]",71610-121,prescription/20180906_dee6139c-4102-4b25-84c0-...,"[{'name': 'TABLET, FILM COATED', 'code': 'C429...","[{'name': 'ORAL', 'code': 'C38288'}]",[],ADVERSE REACTIONS\nAssociated with Discontinua...,NaN,NaN,NaN,NaN,NaN,NaN
46636,46636,SpecGx LLC,Amphetamine Sulfate,"[{'name': 'Amphetamine', 'code': 'CK833KGX7E'}]",0406-1219,prescription/20220309_00242264-40d8-4267-a91a-...,"[{'name': 'TABLET', 'code': 'C42998'}]","[{'name': 'ORAL', 'code': 'C38288'}]",[],ADVERSE REACTIONS\nCardiovascular\nPalpitation...,INDICATIONS AND USAGE\nAmphetamine sulfate tab...,NaN,NaN,NaN,NaN,NaN
46637,46637,"Vensun Pharmaceuticals, Inc.",Sildenafil Citrate,"[{'name': 'SILDENAFIL', 'code': '3M7OB98Y7H'}]",42543-005,prescription/20180612_a3752548-1472-48b6-9998-...,"[{'name': 'TABLET, FILM COATED', 'code': 'C429...","[{'name': 'ORAL', 'code': 'C38288'}]",['\xa0 Mortality with pediatric use \n[see \nW...,6. ADVERSE REACTIONS\nMost common adverse reac...,1. INDICATIONS AND USAGE\nSildenafil tablets a...,NaN,NaN,NaN,NaN,NaN


# Matching avec ID OCRx

In [23]:
df.columns

Index(['Unnamed: 0', 'author', 'drug', 'active_ingredient', 'code', 'filename',
       'form', 'roa', 'adverse_events_list', 'adverse_events', 'indications',
       'active_ingredients', 'numerator_name', 'numerator_val',
       'denominator_name', 'denominator_val'],
      dtype='object')

In [24]:
columns_to_convert = ['form','roa','active_ingredient']
def parse(el):
    if type(el) == str:
        return ast.literal_eval(el)
    return []
for column in columns_to_convert:
    df[column] = df[column].apply(parse)

NameError: name 'ast' is not defined

In [ ]:
from pprint import pprint
import re
from os.path import join as pjoin
import pickle


def gather_medical_term(medical_term,n=10):
    all_els = set()
    for _, row in df.dropna(subset = [medical_term]).iterrows():
        for el in row[medical_term]:
#             import ipdb; ipdb.set_trace()
            if type(el) != dict:
                continue
            name = el['name']
            code = el['code']
            all_els.add((name,code))
    
    print(f"There are {len(all_els)} elements of type {medical_term}")
    print(f"Preview of top {n} samples")
    pprint(list(all_els)[:n])
    return all_els

def gather_ocrx_medical_term(medical_term):
    df = pd.read_csv(f"ocrx-data/{medical_term}_en.csv")
    labels = df['sLabel'].values
    codes = df['s'].values
    return [(str(val),str(val_code)) for val, val_code in zip(labels,codes)]

terms_ocrx = [
    "form",
    "roa",
    "substance"
]
terms_dailymed = ["form","roa","active_ingredient"]
def read_ocrx_data(fn="ocrx-data/mapping_dict.pickle"):
    with open(fn,"rb") as f:
        return pickle.load(f)

In [ ]:
import pandas as pd

# form_list = gather_medical_term('form')
# roa_list = gather_medical_term('roa')
# drugs_list = gather_medical_term('active_ingredient')
dailymed_form, dailymed_roa, dailymed_drugs = [gather_medical_term(term) for term in terms_dailymed]
ocrx_form, ocrx_roa, ocrx_drugs = [gather_ocrx_medical_term(term) for term in terms_ocrx]
# all_needed_terms = {*form_list,*roa_list,*drugs_list}
ocrx_data = read_ocrx_data()

In [ ]:
df.loc[df['active_ingredient_code'] == 'P7MU9UTP52']['filename'].iloc[0]

In [ ]:
for _,row in df.loc[[("NIACIN" in str(val)) for val in df['active_ingredient'].values]][['active_ingredient','active_ingredient_code','filename','drug']].head().iterrows():
    print(len(row['active_ingredient'].split('|||')))
    print(len(row['active_ingredient_code'].split('|||')))
    print(row['active_ingredient'])
    print(row['active_ingredient_code'])

In [ ]:
# foo.label == 'NIACIN'

In [ ]:
# foo = pd.DataFrame(dailymed_drugs,columns=['label','code'])
# foo[foo.label == 'NIACIN']

In [65]:
pairings = [
    [dailymed_form,ocrx_form],
    [dailymed_roa,ocrx_roa],
    [dailymed_drugs,ocrx_drugs]
]
names = ['form','roa','active_ingredient']
match_dfs = []
for name, (dailymed_pair, ocrx_pair) in zip(names,pairings):
    match_df = pd.DataFrame(dailymed_pair,columns=['label','code_dailymed']).merge(pd.DataFrame(ocrx_pair,columns=['label','code_ocrx']),on="label",how="outer")
    match_df.to_csv(f"results/new_match_{name}.csv")
    match_dfs.append(match_df)

In [66]:
active_df = match_dfs[1]

In [68]:
active_df

,label,code_dailymed,code_ocrx
0,IRRIGATION,C38281,4100000048
1,TRANSDERMAL,C38305,4100000083
2,SUBCONJUNCTIVAL,C38298,NaN
3,NOT APPLICABLE,C48623,NaN
4,INTRAPLEURAL,C38259,4100000038
...,...,...,...
121,PERIARTICULAR,NaN,4100000080
122,TEAT DILATOR,NaN,4100000067
123,INTRADISCAL,NaN,4100000064
124,INTUBING,NaN,4100000089


In [71]:
dailymed_roa

{('AURICULAR (OTIC)', 'C38192'),
 ('BUCCAL', 'C38193'),
 ('CONJUNCTIVAL', 'C38194'),
 ('CUTANEOUS', 'C38675'),
 ('DENTAL', 'C38197'),
 ('ENDOSINUSIAL', 'C38206'),
 ('ENDOTRACHEAL', 'C38208'),
 ('ENTERAL', 'C38209'),
 ('EPIDURAL', 'C38210'),
 ('EXTRACORPOREAL', 'C38212'),
 ('HEMODIALYSIS', 'C38200'),
 ('INFILTRATION', 'C38215'),
 ('INTERSTITIAL', 'C38219'),
 ('INTRA-ARTERIAL', 'C38222'),
 ('INTRA-ARTICULAR', 'C38223'),
 ('INTRABRONCHIAL', 'C38225'),
 ('INTRACAMERAL', 'C64984'),
 ('INTRACANALICULAR', 'C132737'),
 ('INTRACARDIAC', 'C38227'),
 ('INTRACAUDAL', 'C38229'),
 ('INTRACAVERNOUS', 'C38230'),
 ('INTRACAVITARY', 'C38231'),
 ('INTRACORONARY', 'C38218'),
 ('INTRADERMAL', 'C38238'),
 ('INTRAEPIDERMAL', 'C38243'),
 ('INTRALESIONAL', 'C38250'),
 ('INTRALYMPHATIC', 'C38252'),
 ('INTRAMEDULLARY', 'C38253'),
 ('INTRAMUSCULAR', 'C28161'),
 ('INTRAOCULAR', 'C38255'),
 ('INTRAPERITONEAL', 'C38258'),
 ('INTRAPLEURAL', 'C38259'),
 ('INTRASINAL', 'C38262'),
 ('INTRASPINAL', 'C38263'),
 ('INTRASYN

In [72]:
match_df

,label,code_dailymed,code_ocrx
0,ROPINIROLE,030PYR8953,3000000129
1,PRAVASTATIN,KXO2KT9N0G,3100005232
2,ROMIDEPSIN,CX3T89XQBK,3000002115
3,DEUTETRABENAZINE,P341G6W9NB,NaN
4,TETRADECYL HYDROGEN SULFATE (ESTER),6326W0DRHY,NaN
...,...,...,...
31919,ALUMINIUM MAGNESIUM HYDROXIDE CARBONATE,NaN,3100005326
31920,ALUMINUM MAGNESIUM HYDROXIDE CARBONATE,NaN,3100005326
31921,"HYDROTALCITE, SYNTHETIC (ANHYDROUS)",NaN,3100005326
31922,MAGNESIUM ALUMINUM HYDROXY CARBONATE (MG6AL2(O...,NaN,3100005326


In [122]:
# lookup_ocrx = {str(row['sLabel']):str(row['s']) for _, row in ocrx_data.iterrows()}
lookup_ocrx = ocrx_data

In [40]:
pd.DataFrame(ocrx_data.items())

,0,1
0,SUBCUTANEOUS,4100000026
1,PERCUTANEOUS,4100000032
2,INTRADERMAL,4100000068
3,INTRAVENOUS,4100000079
4,INTRA-ARTERIAL,4100000091
...,...,...
35504,FOAM,4000000121
35505,JAM,4000000122
35506,SPONGE,4000000123
35507,POWDER FOR SOLUTION,4000000124


In [41]:
terms_dailymed

['form', 'roa', 'active_ingredient']

In [42]:
df

,Unnamed: 0,author,drug,active_ingredient,active_ingredient_code,code,filename,form_code,form,roa,roa_code,adverse_events_list,adverse_events,indications,numerator_name,numerator_val,denominator_name,denominator_val
0,0,Mylan Pharmaceuticals Inc.,NORETHINDRONE AND ETHINYL ESTRADIOL AND FERROU...,ETHINYL ESTRADIOL ||| NORETHINDRONE,423D2T571U ||| T18F433X4S,0378-7308,prescription/20180315_c89388f0-ffe4-4058-a644-...,C47916,KIT,ORAL,C38288,['Serious cardiovascular events and smoking \n...,6 ADVERSE REACTIONS\nThe most common adverse r...,1 INDICATIONS AND USAGE\nNorethindrone and eth...,NaN,NaN,NaN,NaN
1,1,Amneal Pharmaceuticals LLC,Entecavir,ENTECAVIR ANHYDROUS,NNU2O4609D,65162-446,prescription/20201229_007a6ad8-4d23-4543-ae48-...,C42931,"TABLET, FILM COATED",ORAL,C38288,[],6 ADVERSE REACTIONS\nThe following adverse rea...,1 INDICATIONS AND USAGE\nEntecavir tablets are...,NaN,NaN,NaN,NaN
2,2,Qualitest Pharmaceuticals,K-EFFERVESCENT,POTASSIUM CATION,295O53K152,0603-4170,prescription/20120920_5dcd492f-cef1-41ba-b5d2-...,C42910,"TABLET, EFFERVESCENT",ORAL,C38288,[],"ADVERSE REACTIONS:\n Abdominal discomfort, di...",INDICATIONS AN USAGE:\n1. For therapeutic use ...,NaN,NaN,NaN,NaN
3,3,"Denton Pharma, Inc. dba Northwind Pharmaceuticals",Labetalol Hydrochloride,LABETALOL,R5H8897N95,70934-751,prescription/20210819_a9c86f3a-62f7-418c-e053-...,C42931,"TABLET, FILM COATED",ORAL,C38288,[],NaN,NaN,NaN,NaN,NaN,NaN
4,4,REMEDYREPACK INC.,Irbesartan and Hydrochlorothiazide,IRBESARTAN ||| HYDROCHLOROTHIAZIDE,J0E2756Z7N ||| 0J48LPH2TH,70518-2619,prescription/20220412_47423aa5-974e-4c52-8d51-...,C42931,"TABLET, FILM COATED",ORAL,C38288,[],6 ADVERSE REACTIONS\nMost common adverse event...,1 INDICATIONS AND USAGE\nIrbesartan and hydroc...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46634,46634,"EMD Serono, Inc.",Cetrotide,CETRORELIX,OON1HFZ4BA,44087-1225,prescription/20220205_aca7768e-28a7-4027-b1d8-...,C47916,KIT,SUBCUTANEOUS,C38299,[],ADVERSE REACTIONS\nThe safety of Cetrotide\n (...,INDICATIONS AND USAGE\nCetrotide\n (cetrorelix...,NaN,NaN,NaN,NaN
46635,46635,"Aphena Pharma Solutions - Tennessee, LLC",Mirtazapine,MIRTAZAPINE,A051Q2099Q,71610-121,prescription/20180906_dee6139c-4102-4b25-84c0-...,C42931,"TABLET, FILM COATED",ORAL,C38288,[],ADVERSE REACTIONS\nAssociated with Discontinua...,NaN,NaN,NaN,NaN,NaN
46636,46636,SpecGx LLC,Amphetamine Sulfate,Amphetamine,CK833KGX7E,0406-1219,prescription/20220309_00242264-40d8-4267-a91a-...,C42998,TABLET,ORAL,C38288,[],ADVERSE REACTIONS\nCardiovascular\nPalpitation...,INDICATIONS AND USAGE\nAmphetamine sulfate tab...,NaN,NaN,NaN,NaN
46637,46637,"Vensun Pharmaceuticals, Inc.",Sildenafil Citrate,SILDENAFIL,3M7OB98Y7H,42543-005,prescription/20180612_a3752548-1472-48b6-9998-...,C42931,"TABLET, FILM COATED",ORAL,C38288,['\xa0 Mortality with pediatric use \n[see \nW...,6. ADVERSE REACTIONS\nMost common adverse reac...,1. INDICATIONS AND USAGE\nSildenafil tablets a...,NaN,NaN,NaN,NaN


In [43]:
lookup_ocrx

{'SUBCUTANEOUS': 4100000026,
 'PERCUTANEOUS': 4100000032,
 'INTRADERMAL': 4100000068,
 'INTRAVENOUS': 4100000079,
 'INTRA-ARTERIAL': 4100000091,
 'TOPICAL (HOSPITAL/HC FACILITIES)': 4100000044,
 'INTRA-AMNIOTIC': 4100000066,
 'DISINFECTANT (DOMESTIC)': 4100000088,
 'ORAL': 4100000056,
 'TOPICAL': 4100000030,
 'INTRAMUSCULAR': 4100000057,
 'INTRASYNOVIAL': 4100000090,
 'INTRA-ARTICULAR': 4100000085,
 'INFILTRATION': 4100000042,
 'EPIDURAL': 4100000075,
 'TOPICAL (COMMERCIAL)': 4100000058,
 'BLOCK/INFILTRATION': 4100000043,
 'NASAL': 4100000065,
 'DISINFECTANT (BARN)': 4100000021,
 'TOPICAL (HOUSEHOLD)': 4100000017,
 'DISINFECTANT (HOSPITAL/HC FACILITIES)': 4100000007,
 'BUCCAL': 4100000047,
 'DENTAL': 4100000072,
 'OTIC': 4100000094,
 'INTRABURSAL': 4100000018,
 'DISINFECTANT (INSTITUTIONAL/INDUSTRIAL)': 4100000016,
 'DISINFECTANT (FOOD PREMISES)': 4100000028,
 'DISINFECTANT (MEDICAL INSTRUMENTS)': 4100000005,
 'UDDER WASH': 4100000046,
 'RECTAL': 4100000002,
 'INTRACEREBROVENTRICULAR':

In [44]:
# def process_match
from pytrie import Trie
import re
punc_patt = re.compile('[\(\)\,]')
def get_prefix(trie,term):
    try:
        chars, url = trie.longest_prefix_item(term)
        word = ''.join(chars)
        if len(word) <= 3:
            return None
        return word, url, term
    except KeyError:
        return None

def strip_punctuation(word):
    return punc_patt.sub('',word)

def perform_matching(ocrx_terms,dailymed_data):
    lookup_term = {ocrx_term: lookup_ocrx[ocrx_term] for ocrx_term in ocrx_terms}
    cleaned_lookup_term = {strip_punctuation(term):val for term,val in lookup_term.items()}
    #     print(lookup_term)
    cleaned_to_original_mapping = {strip_punctuation(term):term for term in lookup_term.keys()}
    term_trie = Trie(**cleaned_lookup_term)
    '''returns matched, not matched'''
    els = []
    def make_match_dict(wanted_term):
        if strip_punctuation(wanted_term) in cleaned_lookup_term:
            matched_term = cleaned_to_original_mapping[strip_punctuation(wanted_term)]
            return {"term" : wanted_term,"matched_term" : matched_term, "ocrx_id" : lookup_term[matched_term], "match" : "exact"}
        else:
            vals = get_prefix(term_trie,strip_punctuation(wanted_term))
            if vals is not None:
                raw_matched_term = vals[0]
                matched_term = cleaned_to_original_mapping[raw_matched_term]
                ocrx_id = vals[1]
                return {"term" : wanted_term, "matched_term" : matched_term, "ocrx_id" : ocrx_id, "match" : "prefix"}
            return {"term" : wanted_term, "matched_term" : None, "ocrx_id" : None, "match" : "none"}
    return pd.DataFrame([make_match_dict(wanted_term) for wanted_term in dailymed_data])

dailymed_datasets = [dailymed_form, dailymed_roa, dailymed_drugs]
ocrx_datasets = [ocrx_form, ocrx_roa, ocrx_drugs]

matching_dfs = [perform_matching(ocrx_list,dailymed_list) for ocrx_list, dailymed_list in zip(ocrx_datasets,dailymed_datasets)]
ocrx_matching_dfs = [perform_matching(dailymed_list,ocrx_list) for ocrx_list, dailymed_list in zip(ocrx_datasets,dailymed_datasets)]
match_form_df, match_roa_df, match_drug_df = matching_dfs


def do_analysis(df,title):
    length = df.shape[0]
    matched_dailymed = df[df['match'] != 'none'].shape[0]
    matched_none = length - matched_dailymed
    matched_exact = df[df['match'] == 'exact'].shape[0]
    matched_prefix = df[df['match'] == 'prefix'].shape[0]
    print(title)
    print(f"Total: {length}. Exact Match: {matched_exact}. Prefix Match: {matched_prefix}. No match: {matched_none}. Total matched: {matched_dailymed}.")

KeyError: ('SOLUTION (EXTENDED RELEASE)', '4000000106')

In [ ]:
do_analysis(match_form_df,"Statistics on Dailymed Form")
do_analysis(match_roa_df,"Statistics on Dailymed ROA")
do_analysis(match_drug_df,"Statistics on Dailymed Active Ingredients")

In [15]:
match_form_df.to_csv("results/match_form.csv")
match_roa_df.to_csv("results/match_roa.csv")
match_drug_df.to_csv("results/match_drug.csv")

# Exemple de match

In [25]:
match_drug_df[match_drug_df['matched_term'] == 'ACETAMINOPHEN']
df = pd.read_csv("all_drugs_parsed_with_form_and_roa.csv")

In [29]:
filename = df[df['drug'] == 'ACETAMINOPHEN']['filename'].values[0]
print(filename)
df[df['drug'] == 'ACETAMINOPHEN']


prescription/20220410_dc42ab17-cd30-4d1a-e053-2995a90a86ea/dc42ab17-cd31-4d1a-e053-2995a90a86ea.xml


,Unnamed: 0,author,drug,active_ingredient,code,filename,form_code,form,roa,roa_code
35309,35309,"HF Acquisition Co LLC, DBA HealthFirst",ACETAMINOPHEN,ACETAMINOPHEN,51662-1540,prescription/20220410_dc42ab17-cd30-4d1a-e053-...,C42945,"INJECTION, SOLUTION",INTRAVENOUS,C38276


In [ ]:
# Trouver indications therapeutiques (from <item>) (si c'est structure, garder en liste). Sinon

# Trouver effets secondaires (faire une liste structuree si le texte est court)
# Documtner. Combien de substances? Combien de route?

In [30]:
!code prescription/20220410_dc42ab17-cd30-4d1a-e053-2995a90a86ea/dc42ab17-cd31-4d1a-e053-2995a90a86ea.xml

In [1]:
import pandas as pd
pd.read_csv("all_drugs_ade_indications.csv")

,Unnamed: 0,author,drug,active_ingredient,active_ingredient_code,code,filename,form_code,form,roa,roa_code,adverse_events_list,adverse_events,indications
0,0,Mylan Pharmaceuticals Inc.,NORETHINDRONE AND ETHINYL ESTRADIOL AND FERROU...,NORETHINDRONE ||| ETHINYL ESTRADIOL,T18F433X4S ||| 423D2T571U,0378-7308,prescription/20180315_c89388f0-ffe4-4058-a644-...,C47916,KIT,ORAL,C38288,['Serious cardiovascular events and smoking \n...,6 ADVERSE REACTIONS\nThe most common adverse r...,1 INDICATIONS AND USAGE\nNorethindrone and eth...
1,1,Amneal Pharmaceuticals LLC,Entecavir,ENTECAVIR ANHYDROUS,NNU2O4609D,65162-446,prescription/20201229_007a6ad8-4d23-4543-ae48-...,C42931,"TABLET, FILM COATED",ORAL,C38288,[],6 ADVERSE REACTIONS\nThe following adverse rea...,1 INDICATIONS AND USAGE\nEntecavir tablets are...
2,2,Qualitest Pharmaceuticals,K-EFFERVESCENT,POTASSIUM CATION,295O53K152,0603-4170,prescription/20120920_5dcd492f-cef1-41ba-b5d2-...,C42910,"TABLET, EFFERVESCENT",ORAL,C38288,[],"ADVERSE REACTIONS:\n Abdominal discomfort, di...",INDICATIONS AN USAGE:\n1. For therapeutic use ...
3,3,"Denton Pharma, Inc. dba Northwind Pharmaceuticals",Labetalol Hydrochloride,LABETALOL,R5H8897N95,70934-751,prescription/20210819_a9c86f3a-62f7-418c-e053-...,C42931,"TABLET, FILM COATED",ORAL,C38288,[],NaN,NaN
4,4,REMEDYREPACK INC.,Irbesartan and Hydrochlorothiazide,HYDROCHLOROTHIAZIDE ||| IRBESARTAN,J0E2756Z7N ||| 0J48LPH2TH,70518-2619,prescription/20220412_47423aa5-974e-4c52-8d51-...,C42931,"TABLET, FILM COATED",ORAL,C38288,[],6 ADVERSE REACTIONS\nMost common adverse event...,1 INDICATIONS AND USAGE\nIrbesartan and hydroc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46634,46634,"EMD Serono, Inc.",Cetrotide,CETRORELIX,OON1HFZ4BA,44087-1225,prescription/20220205_aca7768e-28a7-4027-b1d8-...,C47916,KIT,SUBCUTANEOUS,C38299,[],ADVERSE REACTIONS\nThe safety of Cetrotide\n (...,INDICATIONS AND USAGE\nCetrotide\n (cetrorelix...
46635,46635,"Aphena Pharma Solutions - Tennessee, LLC",Mirtazapine,MIRTAZAPINE,A051Q2099Q,71610-121,prescription/20180906_dee6139c-4102-4b25-84c0-...,C42931,"TABLET, FILM COATED",ORAL,C38288,[],ADVERSE REACTIONS\nAssociated with Discontinua...,NaN
46636,46636,SpecGx LLC,Amphetamine Sulfate,Amphetamine,CK833KGX7E,0406-1219,prescription/20220309_00242264-40d8-4267-a91a-...,C42998,TABLET,ORAL,C38288,[],ADVERSE REACTIONS\nCardiovascular\nPalpitation...,INDICATIONS AND USAGE\nAmphetamine sulfate tab...
46637,46637,"Vensun Pharmaceuticals, Inc.",Sildenafil Citrate,SILDENAFIL,3M7OB98Y7H,42543-005,prescription/20180612_a3752548-1472-48b6-9998-...,C42931,"TABLET, FILM COATED",ORAL,C38288,['\xa0 Mortality with pediatric use \n[see \nW...,6. ADVERSE REACTIONS\nMost common adverse reac...,1. INDICATIONS AND USAGE\nSildenafil tablets a...
